## Novel dataset web scraper

Gathers novel information from novelupdates, http://www.novelupdates.com/,
then cleans the data and arrange everything into a dataset.
The dataset is finally saved as a csv file.

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

novel_list_page = "http://www.novelupdates.com/novelslisting/?st=1&pg="
novel_page = "http://www.novelupdates.com/?p="

There do not seem to be an easy way to get all novel ids. Therefore, these are gathered from existing list of novels. First the maximum number of novel pages is retrieved and then the novels on these are iterated to get the ids.

In [7]:
# Get the maximum number of pages with novel
def get_novel_list_max_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    dig_pag = soup.find('div', attrs={'class':'digg_pagination'})
    page_links = dig_pag.find_all('a')
    last_page_link = str(page_links[2]) # The last page is the 3rd
    num = re.search('pg=\d+', last_page_link).group()[3:]
    return int(num)

page = requests.get(novel_list_page + '1')
novels_max_pages = get_novel_list_max_pages(page)
print("Pages with novels: " + str(novels_max_pages))

# For testing
#novels_max_pages = 2

# Get all novel ids from the novel lists
def get_novel_ids(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'id':'myTable'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    novel_ids = []
    for row in rows:
        col = row.find_all('td')[-1]
        novel_id = col.a['id'][3:]
        novel_ids.append(novel_id)
    return novel_ids

all_novel_ids = []
for i in range(1,novels_max_pages+1):
    page = requests.get(novel_list_page + str(i))
    novel_ids = get_novel_ids(page)
    all_novel_ids.extend(novel_ids)
    time.sleep(1)

df = pd.DataFrame(all_novel_ids, columns=['id'])

Pages with novels: 129


In [8]:
def parse_novel_page(id):
    page = requests.get(novel_page + str(id))
#    page = requests.get(id) # TEMP
    soup = BeautifulSoup(page.content, 'html.parser')
    wrapper = soup.find('div', attrs={'class': 'wpb_wrapper'})
    
    if wrapper is None:
        return pd.Series()
    data = {}
    
    # generatl information
    #name = 
    #assoc_names = 
    #author = 
    #org_langauge =
    data['genres'] = [genre.text
              for genre in wrapper
                .find('div', attrs={'id': 'seriesgenre'})
                .find_all('a', attrs={'class': 'genre'})]
    #tags =
    
    # publisher
    #start_year = 
    #licensed = 
    #original_publisher =
    #english_publisher =
    
    # chapters
    #volumes_total =
    #chapters_total =
    #chapters_translated =
    #complete
    #complete_translated =
    
    # translated releases
    #release_freq = 
    #activity_week_rank =
    #activity_month_rank =
    #activity_all_time_rank =
    
    # community
    #on_reading_lists =
    #reading_list_month_rank =
    #reading_list_all_time_rank = 

    rating_text = wrapper.find('span', attrs={'class': 'uvotes'}).text.split(' ')
    data['rating'] = float(rating_text[0][1:])
    data['rating_votes'] = int(rating_text[3])
    
    
    # relations
    #related_series_ids =
    #recommended_series_ids =
    
    time.sleep(1)
    return pd.Series(data)

#https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns
#parse_novel_page('http://www.novelupdates.com/series/i-shall-seal-the-heavens/')

#df.merge(df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
df = pd.merge(df, df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
df.to_csv('novels.csv', header=True, index=False)

1173
14845
1175
1177
13038
1171
2042
6612
11133
9343
8233
7959
2696
13155
2742
7145
5400
9314
10927
14629
1179
12408
13080
14794
1069
13197
12584
1181
1183
3992
1185
8674
10925
9469
3244
13580
780
7221
9838
1187
2310
829
10914
1189
8423
3567
14718
12735
12859
11499
8512
14362
7756
399
2151
8074
13558
7151
564
4300
9038
2085
11393
11677
13384
2693
14759
14479
13576
2765
9120
7591
13207
10394
2568
2366
2771
11115
7160
3729
12576
6807
8694
5996
14881
3175
5844
9081
1190
12210
13553
12451
8776
1192
13279
4522
11987
1194
4603
4423
8545
7077
10541
8301
9291
1124
1196
2758
11764
14233
12665
9241
10642
572
9280
5795
4478
3562
9048
14
10802
11014
5131
12620
14942
4859
1198
4869
1200
5245
623
13742
14356
3514
1202
10200
9880
12334
8264
1204
4898
6287
11035
14076
9196
4130
5570
4083
3297
712
1206
7162
1208
8156
8085
1210
7587
5267
7709
9824
6906
10841
14938
595
908
750
1212
4628
7096
14175
1214
9493
7797
13375
5459
3275
13167
2363
10881
142
12598
5374
7133
3437
14584
7375
1216
14641
7668
14851
59

691
14341
6403
7875
732
1641
6946
6451
5472
12745
6386
10322
12586
434
2147
7648
354
7365
3527
189
5763
12122
11168
4188
12295
14850
11353
3272
1643
14815
14491
8820
12106
7934
7859
8514
5161
9101
1645
1647
1649
1651
11073
1653
9096
7113
12330
12866
4728
1655
474
11367
1657
14524
169
1130
895
2948
8289
12104
14009
501
1659
12748
14126
1661
14859
6652
12489
14329
10795
13052
4293
13212
1663
2435
11811
14077
12067
3435
1664
528
1666
6930
3962
1668
602
1670
6179
14486
3853
13139
11422
14731
819
2509
4843
10549
8938
4073
3453
14406
6467
4028
10619
655
5347
7750
6574
7722
3063
3538
6253
11430
12818
6077
10853
4793
4766
12485
13664
5318
14754
3994
7483
12574
7901
8176
13431
13990
6185
10836
5235
8861
13979
2921
12993
513
11708
11528
14876
4288
10938
13749
12811
416
14780
3885
10144
1672
1674
2643
6137
6140
1676
9509
8929
12082
926
429
1678
7171
5298
5566
3503
1680
1682
2612
173
12484
9295
4285
9276
13453
8462
13148
10356
9114
14264
5273
5729
3207
1684
138
10885
1686
8719
1688
5453
2554
1690


3641
3927


In [ ]:
Error: 
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-7-75188395bf04> in <module>()
     53 
     54 #df.merge(df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
---> 55 df = pd.merge(df, df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
     56 df.to_csv('novels.csv', header=True, index=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/series.py in apply(self, func, convert_dtype, args, **kwds)
   2353             else:
   2354                 values = self.asobject
-> 2355                 mapped = lib.map_infer(values, f, convert=convert_dtype)
   2356 
   2357         if len(mapped) and isinstance(mapped[0], Series):

pandas/_libs/src/inference.pyx in pandas._libs.lib.map_infer (pandas/_libs/lib.c:66645)()

<ipython-input-7-75188395bf04> in <lambda>(x)
     53 
     54 #df.merge(df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
---> 55 df = pd.merge(df, df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
     56 df.to_csv('novels.csv', header=True, index=True)

<ipython-input-7-75188395bf04> in parse_novel_page(id)
      9 
     10     genres = [genre.text
---> 11               for genre in wrapper
     12                 .find('div', attrs={'id': 'seriesgenre'})
     13                 .find_all('a', attrs={'class': 'genre'})]

AttributeError: 'NoneType' object has no attribute 'find'